In [1]:
# Library Imports.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Allows plots to appear directly in the notebook.
%matplotlib inline

from patsy import dmatrices
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score 

import pickle

In [2]:
# Read CSV files into Data Frames:
weather = pd.read_csv('working_weatherHistoryResampled_asof_2022.04.06.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)
station = pd.read_csv('working_stationStateResampled_asof_2022.04.06.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)

In [3]:
weather.shape

(684, 7)

In [4]:
station.shape

(75240, 6)

In [5]:
weather.head()

,weatherHour,latitude,longitude,description,temp,humidity,wind_speed
0,2022-02-22 12:00:00,53.35,-6.26,few clouds,282.81,71,10.80
1,2022-02-24 10:00:00,53.35,-6.26,scattered clouds,276.31,80,9.77
2,2022-03-01 09:00:00,53.35,-6.26,clear sky,277.21,86,0.98
3,2022-03-01 10:00:00,53.35,-6.26,scattered clouds,279.13,83,1.78
4,2022-03-01 11:00:00,53.35,-6.26,broken clouds,280.61,80,3.65


In [6]:
station.head()

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes
0,143661,1,2022-02-22 12:00:00,OPEN,12,18
1,143662,2,2022-02-22 12:00:00,OPEN,18,2
2,143663,3,2022-02-22 12:00:00,OPEN,20,13
3,143664,4,2022-02-22 12:00:00,OPEN,22,13
4,143665,5,2022-02-22 12:00:00,OPEN,38,2


In [7]:
stationWeather = pd.merge(station, weather, how="left", on=["weatherHour"])

In [8]:
stationWeather.head()

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed
0,143661,1,2022-02-22 12:00:00,OPEN,12,18,53.35,-6.26,few clouds,282.81,71,10.8
1,143662,2,2022-02-22 12:00:00,OPEN,18,2,53.35,-6.26,few clouds,282.81,71,10.8
2,143663,3,2022-02-22 12:00:00,OPEN,20,13,53.35,-6.26,few clouds,282.81,71,10.8
3,143664,4,2022-02-22 12:00:00,OPEN,22,13,53.35,-6.26,few clouds,282.81,71,10.8
4,143665,5,2022-02-22 12:00:00,OPEN,38,2,53.35,-6.26,few clouds,282.81,71,10.8


In [9]:
stationWeather.tail()

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed
75235,218896,106,2022-04-04 10:00:00,OPEN,3,17,53.35,-6.26,broken clouds,285.57,86,10.84
75236,218897,107,2022-04-04 10:00:00,OPEN,29,11,53.35,-6.26,broken clouds,285.57,86,10.84
75237,218898,108,2022-04-04 10:00:00,OPEN,37,3,53.35,-6.26,broken clouds,285.57,86,10.84
75238,218899,109,2022-04-04 10:00:00,OPEN,4,24,53.35,-6.26,broken clouds,285.57,86,10.84
75239,218900,110,2022-04-04 10:00:00,OPEN,28,2,53.35,-6.26,broken clouds,285.57,86,10.84


In [10]:
stationWeather.dtypes

ID                         int64
stationId                  int64
weatherHour               object
status                    object
available_bike_stands      int64
available_bikes            int64
latitude                 float64
longitude                float64
description               object
temp                     float64
humidity                   int64
wind_speed               float64
dtype: object

In [11]:
stationWeather[['weatherDate', 'weatherHour']] = stationWeather['weatherHour'].str.split(' ', expand=True)
stationWeather

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed,weatherDate
0,143661,1,12:00:00,OPEN,12,18,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22
1,143662,2,12:00:00,OPEN,18,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22
2,143663,3,12:00:00,OPEN,20,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22
3,143664,4,12:00:00,OPEN,22,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22
4,143665,5,12:00:00,OPEN,38,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75235,218896,106,10:00:00,OPEN,3,17,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04
75236,218897,107,10:00:00,OPEN,29,11,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04
75237,218898,108,10:00:00,OPEN,37,3,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04
75238,218899,109,10:00:00,OPEN,4,24,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04


In [12]:
stationWeather[['weatherHour', 'weatherMinute', 'weatherSecond']] = stationWeather['weatherHour'].str.split(':', expand=True)
stationWeather

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed,weatherDate,weatherMinute,weatherSecond
0,143661,1,12,OPEN,12,18,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00
1,143662,2,12,OPEN,18,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00
2,143663,3,12,OPEN,20,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00
3,143664,4,12,OPEN,22,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00
4,143665,5,12,OPEN,38,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75235,218896,106,10,OPEN,3,17,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00
75236,218897,107,10,OPEN,29,11,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00
75237,218898,108,10,OPEN,37,3,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00
75238,218899,109,10,OPEN,4,24,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00


In [13]:
stationWeather['cal_bike_stands'] = stationWeather.available_bike_stands + stationWeather.available_bikes
stationWeather

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed,weatherDate,weatherMinute,weatherSecond,cal_bike_stands
0,143661,1,12,OPEN,12,18,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,30
1,143662,2,12,OPEN,18,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,20
2,143663,3,12,OPEN,20,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,33
3,143664,4,12,OPEN,22,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,35
4,143665,5,12,OPEN,38,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75235,218896,106,10,OPEN,3,17,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,20
75236,218897,107,10,OPEN,29,11,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,40
75237,218898,108,10,OPEN,37,3,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,40
75238,218899,109,10,OPEN,4,24,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,28


In [14]:
# Change description to category type for numeric encoding
stationWeather['description'] = stationWeather['description'].astype('category')
stationWeather.dtypes

ID                          int64
stationId                   int64
weatherHour                object
status                     object
available_bike_stands       int64
available_bikes             int64
latitude                  float64
longitude                 float64
description              category
temp                      float64
humidity                    int64
wind_speed                float64
weatherDate                object
weatherMinute              object
weatherSecond              object
cal_bike_stands             int64
dtype: object

In [15]:
# Catergorical Encoding
stationWeather['num_desc'] = stationWeather['description'].cat.codes
stationWeather

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed,weatherDate,weatherMinute,weatherSecond,cal_bike_stands,num_desc
0,143661,1,12,OPEN,12,18,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,30,2
1,143662,2,12,OPEN,18,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,20,2
2,143663,3,12,OPEN,20,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,33,2
3,143664,4,12,OPEN,22,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,35,2
4,143665,5,12,OPEN,38,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,40,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75235,218896,106,10,OPEN,3,17,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,20,0
75236,218897,107,10,OPEN,29,11,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,40,0
75237,218898,108,10,OPEN,37,3,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,40,0
75238,218899,109,10,OPEN,4,24,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,28,0


In [16]:
# Checking correlations for all the contious features
stationWeather.corr()

,ID,stationId,available_bike_stands,available_bikes,latitude,longitude,temp,humidity,wind_speed,cal_bike_stands,num_desc
ID,1.000000,1.461928e-03,0.000109,-0.002508,NaN,NaN,2.304128e-01,-1.411975e-01,-2.389431e-01,-0.002657,8.602108e-03
stationId,0.001462,1.000000e+00,-0.012042,-0.042931,NaN,NaN,8.552269e-16,-6.187358e-16,-2.477227e-16,-0.064043,2.009989e-16
available_bike_stands,0.000109,-1.204187e-02,1.000000,-0.680484,NaN,NaN,1.946609e-02,-1.353588e-02,1.009944e-03,0.574030,-1.162099e-03
available_bikes,-0.002508,-4.293149e-02,-0.680484,1.000000,NaN,NaN,-2.569532e-02,2.459045e-02,-4.632753e-03,0.209393,-3.387794e-03
latitude,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
longitude,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
temp,0.230413,8.552269e-16,0.019466,-0.025695,NaN,NaN,1.000000e+00,-4.675910e-01,1.368502e-01,-0.002737,-3.380526e-02
humidity,-0.141197,-6.187358e-16,-0.013536,0.024590,NaN,NaN,-4.675910e-01,1.000000e+00,-1.789565e-01,0.009416,-4.910726e-02
wind_speed,-0.238943,-2.477227e-16,0.001010,-0.004633,NaN,NaN,1.368502e-01,-1.789565e-01,1.000000e+00,-0.003829,1.195065e-02
cal_bike_stands,-0.002657,-6.404345e-02,0.574030,0.209393,NaN,NaN,-2.737095e-03,9.415986e-03,-3.829206e-03,1.000000,-5.336483e-03


# Random Forest

In [17]:
X = stationWeather[['stationId', 'weatherHour', 'temp', 'humidity', 'wind_speed', 'cal_bike_stands', 'num_desc']]
y = stationWeather.available_bikes

<h2>Training</h2>

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [19]:
random_forest = RandomForestRegressor(n_estimators = 1000, random_state = 42)
random_forest.fit(X_train, y_train)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [20]:
# Serialize model object into a file called model.pkl on disk using pickle
with open('dwmb_resample_allStation_randomForest_model.pkl', 'wb') as handle:
    pickle.dump(random_forest, handle, pickle.HIGHEST_PROTOCOL)

<h2>Testing</h2>

In [21]:
randomForest_predictions = random_forest.predict(X_test)
print(type(randomForest_predictions))


<class 'numpy.ndarray'>


In [22]:
df = pd.DataFrame({'Actual':y_test, 'Predicted':randomForest_predictions})
df

,Actual,Predicted
47568,0,3.198
62280,1,12.937
15008,9,9.470
34223,30,26.274
64509,25,19.886
...,...,...
32972,11,16.227
38023,30,27.562
61072,11,5.866
60529,17,20.763


In [23]:
# Metrics for the Model
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, randomForest_predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, randomForest_predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, randomForest_predictions)))

Mean Absolute Error: 4.677317384370016
Mean Squared Error: 36.610078208842815
Root Mean Squared Error: 6.050626265837514


In [27]:
# Calculate the absolute errors
errors = abs(randomForest_predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
print('MAPE:', mape)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Mean Absolute Error: 4.68 degrees.
MAPE: 47568            inf
62280    1193.700000
15008       5.222222
34223      12.420000
64509      20.456000
            ...     
32972      47.518182
38023       8.126667
61072      46.672727
60529      22.135294
37864       7.250000
Name: available_bikes, Length: 22572, dtype: float64
Accuracy: -inf %.


In [28]:
# Function to output evaluation metrics
def printMetrics(testActualVal, predictions):
    #classification evaluation measures
    print('\n==============================================================================')
    print("MAE: ", metrics.mean_absolute_error(testActualVal, predictions))
    #print("MSE: ", metrics.mean_squared_error(testActualVal, predictions))
    print("RMSE: ", metrics.mean_squared_error(testActualVal, predictions)**0.5)
    print("R2: ", metrics.r2_score(testActualVal, predictions))

In [29]:
printMetrics(y_test, randomForest_predictions)


MAE:  4.677317384370016
RMSE:  6.050626265837514
R2:  0.535935623407034
